
# Gradient Boosting  for classification with Python


Let's first import the required libraries:


## Objectives

After completing this lab you will be able to:

*   Understand   Gradient Boosting  is a linear combination of  𝑇 weak classifiers
*   Apply Gradient Boosting using  XGBoost,
*   Understand Hyperparameters selection in  XGBoost


In [2]:
import pandas as pd
import pylab as plt
import numpy as np
import scipy.optimize as opt
from sklearn import preprocessing
%matplotlib inline 
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.tree import DecisionTreeRegressor
from tqdm import tqdm

Ignore error warnings.


In [3]:
import warnings
warnings.filterwarnings('ignore')

This function will calculate the accuracy of the training and testing data given a model.


In [4]:
def get_accuracy(X_train, X_test, y_train, y_test, model):
    return  {"test Accuracy":metrics.accuracy_score(y_test, model.predict(X_test)),
             "train Accuracy": metrics.accuracy_score(y_train, model.predict(X_train))}

This function calculates the average accuracy of differnt learning rates on training and test data.


In [5]:
def get_accuracy_boost(X,y,title,times=20,xlabel='Number Estimators',Learning_rate_=[0.2,0.4,0.6,1], n_est = 100):

    lines_array=['solid','--', '-.', ':']

    N_estimators=[n*2 for n in range(1,n_est//2)]
    
    train_acc=np.zeros((times,len(Learning_rate_),len(N_estimators)))
    test_acc=np.zeros((times,len(Learning_rate_),len(N_estimators)))


    #Iterate through different number of Learning rate  and average out the results  
    
    for n in tqdm(range(times)):
        X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3)
        for n_estimators in N_estimators:
            for j,lr in enumerate(Learning_rate_):


                model = XGBClassifier(objective=objective,learning_rate=lr,n_estimators=n_estimators,eval_metric='mlogloss')


                model.fit(X_train,y_train)



                Accuracy=get_accuracy(X_train, X_test, y_train, y_test,  model)



                train_acc[n,j,(n_estimators//2)-1]=Accuracy['train Accuracy']
                test_acc[n,j,(n_estimators//2)-1]=Accuracy['test Accuracy']
    



    fig, ax1 = plt.subplots()
    mean_test=test_acc.mean(axis=0)
    mean_train=train_acc.mean(axis=0)
    ax2 = ax1.twinx()

    for j,(lr,line) in enumerate(zip(Learning_rate_,lines_array)): 

        ax1.plot(mean_train[j,:],linestyle = line,color='b',label="Learning rate "+str(lr))
        ax2.plot(mean_test[j,:],linestyle = line, color='r',label=str(lr))

    ax1.set_ylabel('Training accuracy',color='b')
    ax1.legend()
    ax2.set_ylabel('Testing accuracy', color='r')
    ax2.legend()
    ax1.set_xlabel(xlabel)
    plt.show()

In [7]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

Imagine that you are a medical researcher compiling data for a study. You have collected data about a set of patients, all of whom suffered from the same illness. During their course of treatment, each patient responded to one of 5 medications, Drug A, Drug B, Drug C, Drug X and y.

Part of your job is to build a model to find out which drug might be appropriate for a future patient with the same illness. The features of this dataset are Age, Sex, Blood Pressure, and the Cholesterol of the patients, and the target is the drug that each patient responded to.

It is a sample of multiclass classifier, and you can use the training part of the dataset to build a decision tree, and then use it to predict the class of a unknown patient, or to prescribe a drug to a new patient.


In [8]:
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%203/data/drug200.csv", delimiter=",")
df.head()

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,drugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,drugY


Let's create the X and Y for our dataset:


In [9]:
X = df[['Age', 'Sex', 'BP', 'Cholesterol', 'Na_to_K']].values
X[0:5]

array([[23, 'F', 'HIGH', 'HIGH', 25.355],
       [47, 'M', 'LOW', 'HIGH', 13.093],
       [47, 'M', 'LOW', 'HIGH', 10.114],
       [28, 'F', 'NORMAL', 'HIGH', 7.798],
       [61, 'F', 'LOW', 'HIGH', 18.043]], dtype=object)

In [10]:
y = df["Drug"]
y[0:5]

0    drugY
1    drugC
2    drugC
3    drugX
4    drugY
Name: Drug, dtype: object

Now lets use a <code>LabelEncoder</code> to turn categorical features into numerical:


In [11]:
from sklearn import preprocessing

In [12]:
le_sex = preprocessing.LabelEncoder()
le_sex.fit(['F','M'])
X[:,1] = le_sex.transform(X[:,1]) 


le_BP = preprocessing.LabelEncoder()
le_BP.fit([ 'LOW', 'NORMAL', 'HIGH'])
X[:,2] = le_BP.transform(X[:,2])


le_Chol = preprocessing.LabelEncoder()
le_Chol.fit([ 'NORMAL', 'HIGH'])
X[:,3] = le_Chol.transform(X[:,3]) 

X[0:5]

array([[23, 0, 0, 0, 25.355],
       [47, 1, 1, 0, 13.093],
       [47, 1, 1, 0, 10.114],
       [28, 0, 2, 0, 7.798],
       [61, 0, 1, 0, 18.043]], dtype=object)

Split the data into training and testing data with a 80/20 split:


In [13]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

In [14]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (160, 5) (160,)
Test set: (40, 5) (40,)


We can use GridSearch for Exhaustive search over specified parameter values.


In [15]:
param_grid = {'learning_rate': [0.1*(n+1) for n in range(2)],
             'n_estimators' : [2*n+1 for n in range(2)] }

Create a <code>XGBClassifier</code>object called <cood>model</code>, set `objective` to  `binary:logistic` and `,eval_metric` to `mlogloss` :


In [16]:
# Map the classes 2 and 4 to 0 and 1
label_encoder = LabelEncoder()
y_mapped = label_encoder.fit_transform(y)  # This will convert 2 to 0 and 4 to 1

In [17]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_mapped, test_size=0.2, random_state=4)

In [18]:
# Define the model with binary logistic objective
model = XGBClassifier(objective='binary:logistic', eval_metric='logloss')

Create <code>GridSearchCV</code> object called `search` with the `estimator` set to <code>model</code>, <code>param_grid</code> set to <code>param_grid</code>, <code>scoring</code> set to <code>neg_log_loss</code>, and <code>cv</code> set to 3 and Fit the <code>GridSearchCV</code> object to our <code>X_train</code> and <code>y_train</code> data (this may take some time)


In [19]:

search = GridSearchCV(estimator=model, 
                      param_grid=param_grid,
                      scoring="neg_log_loss")
search.fit(X_train, y_train)

GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='logloss', feature_types=None,
                                     gamma=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             param_grid={'learning_rate': [0.1, 0.2], 'n_estimators': [1, 3]},
             scoring='neg_log_loss')

We can find the accuracy of the best model.


In [20]:
search.best_score_

-0.794052755956755

We can find the best parameter values:


In [21]:
search.best_params_

{'learning_rate': 0.2, 'n_estimators': 3}

We can find the accuracy test data:


In [22]:
print(get_accuracy(X_train, X_test, y_train, y_test, search.best_estimator_))

{'test Accuracy': 0.85, 'train Accuracy': 1.0}


## How  Gradient Boosting  Works (Optional)


Let's try to sketch   how Gradient Boosting works like any supervised problem we have a dataset  ${(x_1, y_1), ..., (x_N,; y_N)} $, the strong classifier $H_{T}(x)$  is a linear combination of $T$  weak classifiers $h_t(x)$  usually trees and $\alpha_t$  is a constant in many cases $\alpha_t=1$  . Although each classifier $h_t(x)$ appears independent, the  contains information about the error of classifiers from $h_1(x),.., h_{t}(x)$.

$H_{T}(\mathbf{x}) =   \sum_{t=1}^T \alpha_t h_t(\mathbf{x}) $

Borrowing the notation form <a href=https://www.cs.cornell.edu/courses/cs4780/2018fa/lectures/lecturenote19.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML241ENSkillsNetwork31576874-2022-01-01> Kilian Weinberger </a>, we have a the cost function:

$\ell(H)=rac{1}{n}\sum_{i=1}^n \ell(H_{T}(\mathbf{x}_i),y_i)$

Where $\ell$ convex i.e bole shape shown below and differentiable, such as derivative exists.


If you're familiar with cost functions, this may seem strange as we are dealing with a function of $H$ not of parameters. Ideally would like to minimize the cost for  $h_{T}(\mathbf{x})$  and  $H_{T-1}(\mathbf{x})$ at the same time   , but this is difficult. So we minimize the cost with respect to   $h_{T}(\mathbf{x})$  while keeping $H_{T-1}(\mathbf{x})$ fixed; not only is this simpler it prevents overfitting.

$h_{t}(x) = 	extrm{argmin}_{   h \in \mathbb{H}} \sum_{i=1}^n \ell(  H_{t-1}(\mathbf{x}_i) + \gamma h_t(\mathbf{x}_i),y_i)$  (1)


Usually $\gamma$  is selected using validation data, but in some cases, like AdaBoost, you can find  the expression for the  optimal value of $\gamma$.


### How to Minimize Cost


We would like to find the value of classifier $h_{t}(x)$ in red  that minimises the Cost function in (1), this is difficult so we minimize the first order Taylor Approximation as shown in (2). We see that it is only a function of the second term in red.


$\ell(H+\alpha \color{red}{h})\approx\ell(H)+ \gamma \sum_{i = 1}^{n} r_{i,t-1} \color{red}{h_{t}(\mathbf{x}_i)}$ (2)

$r_{i,t-1}=rac{\partial \ell}{\partial [H_{t-1}(\mathbf{x}_i)]}$


An example of Taylor Approximation is shown in the following figure, the true cost function is in blue and the first Taylor Approximation is in Orange.


For small values of $\gamma$ this will hold, using the approximation, we can decouple the problem as follows; the $argmax$ of $h$  only cares about the red terms. As a result, we find the value of $r_{i}$ and the minimum concerning $h$. First let's find $r_{i}$, we need the loss function:


$h_{t}(x) = 	extrm{argmin}_{   h \in \mathbb{H}}  \sum_{i=1}^n \ell(  H_{t-1}(\mathbf{x}_i) + \gamma h_{t}(\mathbf{x}_i),y_i) $
$ \approx 	extrm{argmin}_{   h \in \mathbb{H}} \ell(H)+  \sum_{i = 1}^{n} r_{i} \color{red}{h_{t}(\mathbf{x}_i)} $


$=	extrm{argmin}_{h \in \mathbb{H}}\{ \sum_{i = 1}^{n} r_{i} \color{red}{h_{T}(\mathbf{x}_i)}\}$  (3)


### Example with Python


This all seems a little confusing so let’s do a simple example with the Root mean square error loss are write the Python code. This is not usually used for regression but can be used for classification and  the math is relatively  simple to understand. First we calculate $r_{i}$:


$ \ell(H) =\sum_{i=1}^n (r_{i,t-1}-  H_{t-1}(\mathbf{x}_i))^{2} $

$ r_{i,t}=rac{\partial \ell}{\partial [H_{T-1}(\mathbf{x}_i)]}=2\sum_{i=1}^n ( r_{i,t-1}-  H_{T-1}(\mathbf{x}_i))$


Then we plug it into equation 3; with some math, we get the following expression below, just a note these steps are not as simple as they look, so check out  <a href=https://www.cs.cornell.edu/courses/cs4780/2018fa/lectures/lecturenote19.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML241ENSkillsNetwork31576874-2022-01-01> Kilian Weinberger </a> for more details.


$h_{t}(x) =	extrm{argmin}_{h \in \mathbb{H}}{ \sum_{i = 1}^{n} r_{i,t} \color{red}{h_{t}(\mathbf{x}_i)}}$

$=	extrm{argmin}_{h \in \mathbb{H}}{ \sum_{i = 1}^{n} 2\sum_{i=1}^n (y_i-  H_{T-1}(\mathbf{x}_i)) \color{red}{h_{T}(\mathbf{x}_i)}}$

$=	extrm{argmin}_{h \in \mathbb{H}}{ \sum_{i = 1}^{n} (r_{i,t}^{2}- {h_{t}(\mathbf{x}_i)})^{2}}$


Let’s go through a few iterations; at the same time implement the algorithm with  Python. For the Python portion we will use the Toy data where the class of y is as following $y=0$ if $ 0<x<1$ else if  $y=1$, $1 \leq x< 2$ else $y=3$.


For the first iteration we start off with $h_1(\mathbf{x})$ minimizing the original labels $y_{i}$ that equals $r_{i,1}$.


$h_{1}(x)=	extrm{argmin}_{h \in \mathbb{H}}{ \sum_{i = 1}^{n} (y_{i}- {h_{1}(\mathbf{x}_i)})^{2}}$


All this means is we apply any algorithm with the cost function in the same form to the data. One constraint is  $h$ has to be non-linear, in Python:


We can plot the prediction, we see it's not even a true classifier.


For the second step  we set  $H_{1}(\mathbf{x})=h_{1}(\mathbf{x})$,then we calculate $r_{i,2}$.


$ r_{i,2}=( r_{i,1}-  H_{1}(\mathbf{x}_i))$


We then find the second learner $h_{2}(\mathbf{x})$ by minimizing


$h_{2}(x)=	extrm{argmin}_{h \in \mathbb{H}}{ \sum_{i = 1}^{n} (r_{i,2}^{2}- {h_{2}(\mathbf{x}_i)})^{2}}$


We then update $H_{2} (\mathbf{x})= H_{1} (\mathbf{x})+ \gamma h_{2}(\mathbf{x})$


We repeat the process for $ t$ times:


$ r_{i,t}^2=( r_{i,t-1}-  H_{t-1}(\mathbf{x}_i))^{2}$


$h_{t}(x)=	extrm{argmin}_{h \in \mathbb{H}}{ \sum_{i = 1}^{n} (r_{i,t}^{2}- {h_{t}(\mathbf{x}_i)})^{2}}$


We then update $H_{t} (\mathbf{x})= H_{t-1} (\mathbf{x}) + \gamma h_{t}(\mathbf{x})$


We can perform the Operation with a loop in Python. First, we will write a function to make a prediction using an input of a list of predictors `weak_learners`.


Then we train a new weak learner recursively setting gamma=1, and plot the results for each iteration.
